# create Fact Table

**Reading Silver Data**

In [0]:
%sql

select * from parquet.`abfss://silver@adlscarsaless.dfs.core.windows.net/carsales`

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit


In [0]:
df_silver = spark.sql("select * from parquet.`abfss://silver@adlscarsaless.dfs.core.windows.net/carsales`")

In [0]:
df_silver.display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit


**Reading all the DIM's (dimensions)**

In [0]:
df_dealer = spark.sql("select * from cars_catalog.gold.dim_dealer")

df_branch = spark.sql("select * from cars_catalog.gold.dim_branch")

df_model = spark.sql("select * from cars_catalog.gold.dim_model")

df_date = spark.sql("select * from cars_catalog.gold.dim_date")

**Bringing Keys to the FACT table**

In [0]:
df_fact = df_silver.join(df_branch,
                         df_silver['Branch_ID']==df_branch['Branch_ID'],
                         how='left')\
    .join(df_dealer,df_silver['Dealer_ID']==df_dealer['Dealer_ID'],how='left')\
    .join(df_model,df_silver['Model_ID']==df_model['Model_ID'],how='left')\
    .join(df_date,df_silver['Date_ID']==df_date['Date_ID'],how='left')\
    .select(
            df_silver['Revenue'],df_silver['Units_Sold'],df_silver['RevPerUnit'],
            df_branch['dim_branch_key'],
            df_dealer['dim_dealer_key'],
            df_model['dim_model_key'],
            df_date['dim_date_key']
        )

### writing FACT table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('factsales'):
    delta_tbl = DeltaTable.forName(spark,'cars_catalog.gold.factsales')

    delta_tbl.alias('trgt').merge(df_fact.alias('src'),"trgt.dim_branch_key==src.dim_branch_key and trgt.dim_dealer_key==src.dim_dealer_key and trgt.dim_model_key==src.dim_model_key and trgt.dim_date_key==src.dim_date_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
        
else:
    df_fact.write.format('delta')\
    .mode('Overwrite')\
    .option("path","abfss://gold@adlscarsaless.dfs.core.windows.net/factsales")\
    .saveAsTable('cars_catalog.gold.factsales')

In [0]:

spark.sql("select * from cars_catalog.gold.factsales")

DataFrame[Revenue: bigint, Units_Sold: bigint, RevPerUnit: double, dim_branch_key: bigint, dim_dealer_key: bigint, dim_model_key: bigint, dim_date_key: bigint]